# Kaggle League of Legends competition - Neural Network Models

## Team: Elden Ring

<img src="https://eldenring.wiki.fextralife.com/file/Elden-Ring/mirel_pastor_of_vow.jpg" alt="PRAISE DOG" style="width:806px;height:600px;"/>

#### PRAISE THE DOG!

## How to Win at League of Legends?

### Uninstall LoL and [install Dota 2](https://store.steampowered.com/app/570/Dota_2/), EZ. (just kidding, both games are great. Volvo pls gib patch.)

<img src = "https://static.wikia.nocookie.net/dota2_gamepedia/images/7/78/Keyart_phoenix.jpg/revision/latest/" alt="SKREE CAW CAW IM A BIRD" style="width:800px;height:497px;">

In [74]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, KFold

from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.preprocessing import MinMaxScaler

In [85]:
X_train_original = pd.read_csv('../data/participants_train.csv')
X_test_original = pd.read_csv('../data/participants_test.csv')
y_train_original = pd.read_csv('../data/train_winners.csv')

champion_mastery = pd.read_csv('../data/champion_mastery.csv')
champion = pd.read_json('../data/champion.json')

team_positions = pd.read_csv('../data/teamPositions.csv')

train_last_frame_values = pd.read_csv('../data/train_last_frame_values.csv')
test_last_frame_values = pd.read_csv('../data/test_last_frame_values.csv')

training_events = pd.read_csv('../data/training_events.csv')
testing_events = pd.read_csv('../data/testing_events.csv')

submission = pd.read_csv('../data/sample_submission.csv')

In [86]:
# function that converts values to negative (for the second team, teamId 200)
# it leaves the first team values, teamId intact

def convert_team_values(df, col_names):
    
    
    for col in col_names:
        df[col] = np.where(df['teamId'] == 200,
                            -1* df[col],
                                df[col])
        
    return

In [87]:
# transformation needed on the dataframes (see the file prediction_models)
vars = ['wards_placed', 'wards_killed', 'turretplates_destroyed', 'elite_monsters_killed']

convert_team_values(training_events, vars)
convert_team_values(testing_events, vars)

training_events = training_events.groupby('matchId')[vars].sum()
testing_events = testing_events.groupby('matchId')[vars].sum()

champion_data = pd.json_normalize(champion['data'])
champion_data['key'] = champion_data['key'].astype(int)

champion_types= champion_data.explode('tags').pivot_table(values='id', index='key', columns='tags', aggfunc='count').fillna(0).reset_index()

In [78]:
# to be used later to measure the accuracy!
kfold = KFold(n_splits = 10, shuffle = True, random_state = 42)

# this is to extract the column that is needed for training
y_train = y_train_original['winner']

## Using Neural Networks now to try and improve the logreg predictions!

In [79]:
variables = ['summonerLevel', 'championLevel','championPoints', 
             'Assassin', 'Fighter', 'Mage', 'Marksman', 'Support', 'Tank',
            'info.attack', 'info.defense', 'info.magic', 'info.difficulty',
            'stats.hpregenperlevel',	'stats.mpregen', 'stats.mpregenperlevel',	'stats.crit',	'stats.critperlevel',
            'stats.attackdamage', 'stats.attackdamageperlevel', 'stats.attackspeedperlevel',	'stats.attackspeed',
            'final_gold', 'final_xp', 'final_abilityhaste', 'final_abilitypower', 'final_armor', 'final_armorpen',
            'final_armorpenpercent', 'final_atkdmg', 'final_bns_armorpenpercent', 'final_bns_magicpenpercent', 'final_ccreduction',
            'final_cdreduction', 'final_remaining_health', 'final_health', 'final_healthrgn', 'final_lifesteal', 'final_mppen',
            'final_mgpenpercent', 'final_mgres', 'final_ms', 'final_omnivamp', 'final_physicalvamp', 'final_power', 'final_powermax',
            'final_powerregen', 'final_spellvamp', 'final_currentgold', 'final_magicdmgdone', 'final_magicdmgdonetochamps', 'final_magicdmgtaken',
            'final_physdmgdone', 'final_physdmgdonetochamps', 'final_physdmgtaken', 'final_dmgdone', 'final_dmgdonetochamps', 'final_dmgtaken', 
            'final_truedmgdone', 'final_truedmgdonetochamps', 'final_truedmgtaken', 'final_goldpersec', 'final_jungleminionskilled', 'final_lvl',
            'final_minionskilled', 'final_jungleminionskilled', 'final_jungleminionskilled', 'final_jungleminionskilled', 'final_enemycontrolled'                  
             ]

X_train = pd.merge(X_train_original, team_positions, how='inner', on=['matchId', 'participantId'])
X_train = pd.merge(X_train, champion_data, how='inner', left_on='championId', right_on='key')
X_train = pd.merge(X_train, champion_types, how='inner', left_on='championId', right_on='key')
X_train = pd.merge(X_train, train_last_frame_values, how='inner', on=['matchId', 'participantId'])
X_train = pd.merge(X_train, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X_train = X_train.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)


convert_team_values(X_train, variables)

X_train_perlane = X_train.groupby(['matchId', 'teamPosition'])[['final_gold']].sum().pivot_table(values='final_gold', index='matchId', columns='teamPosition').reset_index().drop(columns=0)

for lane in ['BOTTOM', 'JUNGLE', 'MIDDLE', 'TOP', 'UTILITY']:
  X_train_perlane[f'{lane}'] = np.where(X_train_perlane[f'{lane}'] >= 0, 1, -1)

X_train = (
    X_train
    .groupby(['matchId'])[variables]
    .sum()
    .reset_index()
)

X_train = pd.merge(X_train, X_train_perlane, how='inner', on='matchId').reset_index(drop = True)
X_train = pd.merge(X_train, training_events, how='inner', on='matchId').reset_index(drop = True)

In [80]:
X_train.head()

,matchId,summonerLevel,championLevel,championPoints,Assassin,Fighter,Mage,Marksman,Support,Tank,...,final_enemycontrolled,BOTTOM,JUNGLE,MIDDLE,TOP,UTILITY,wards_placed,wards_killed,turretplates_destroyed,elite_monsters_killed
0,0,682,0.0,-605428.0,-1.0,-1.0,2.0,0.0,0.0,-2.0,...,67664,-1,-1,1,-1,-1,-64,-1,-2,-1
1,1,628,8.0,1356027.0,3.0,0.0,0.0,0.0,-1.0,-1.0,...,-61783,1,1,1,-1,1,1,0,-1,-1
2,2,1049,1.0,-273911.0,-2.0,1.0,0.0,0.0,0.0,0.0,...,-132630,1,-1,-1,-1,-1,4,1,-2,1
3,3,-1027,-3.0,-287667.0,1.0,1.0,1.0,-1.0,-1.0,0.0,...,-39616,1,1,-1,-1,1,4,0,2,0
4,4,1612,7.0,503668.0,0.0,0.0,2.0,-1.0,0.0,-1.0,...,16629,-1,1,-1,1,1,4,0,-1,-1


In [81]:
pipeline_neuralnetwork = Pipeline(
    steps = [
        ('scaler', MinMaxScaler()),
        ('nn', MLPClassifier(verbose = True,
                             hidden_layer_sizes = (100, 100, 100, 100),
                             activation = 'tanh',
                             max_iter = 10000,
                             alpha=0.05))
    ]
)

In [82]:
pipeline_neuralnetwork.fit(X_train, y_train)

Iteration 1, loss = 0.66849572
Iteration 2, loss = 0.62492647
Iteration 3, loss = 0.62073670
Iteration 4, loss = 0.60617163
Iteration 5, loss = 0.61613392
Iteration 6, loss = 0.60443803
Iteration 7, loss = 0.60497042
Iteration 8, loss = 0.59647355
Iteration 9, loss = 0.59316923
Iteration 10, loss = 0.59304150
Iteration 11, loss = 0.59657358
Iteration 12, loss = 0.59080561
Iteration 13, loss = 0.59035417
Iteration 14, loss = 0.58673575
Iteration 15, loss = 0.59459825
Iteration 16, loss = 0.58465865
Iteration 17, loss = 0.58895721
Iteration 18, loss = 0.58755091
Iteration 19, loss = 0.58328436
Iteration 20, loss = 0.57939311
Iteration 21, loss = 0.58285486
Iteration 22, loss = 0.58145488
Iteration 23, loss = 0.58454298
Iteration 24, loss = 0.57697932
Iteration 25, loss = 0.57648520
Iteration 26, loss = 0.57901413
Iteration 27, loss = 0.57501599
Iteration 28, loss = 0.57388428
Iteration 29, loss = 0.57613449
Iteration 30, loss = 0.57234724
Iteration 31, loss = 0.57186986
Iteration 32, los

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('nn',
                 MLPClassifier(activation='tanh', alpha=0.05,
                               hidden_layer_sizes=(100, 100, 100, 100),
                               max_iter=10000, verbose=True))])

In [83]:
accuracy_score(
    y_true = y_train,
    y_pred = pipeline_neuralnetwork.predict(X_train)
)

0.717625

In [84]:
summoner_cv_scores = cross_val_score(
    estimator = pipeline_neuralnetwork,
    X = X_train,
    y = y_train,
    cv = kfold
)

print(summoner_cv_scores)
print(np.mean(summoner_cv_scores))

Iteration 1, loss = 0.65800709
Iteration 2, loss = 0.61456134
Iteration 3, loss = 0.61665560
Iteration 4, loss = 0.60812177
Iteration 5, loss = 0.60439405
Iteration 6, loss = 0.60015250
Iteration 7, loss = 0.59699050
Iteration 8, loss = 0.59816644
Iteration 9, loss = 0.59363554
Iteration 10, loss = 0.58751001
Iteration 11, loss = 0.59588893
Iteration 12, loss = 0.59765861
Iteration 13, loss = 0.58942774
Iteration 14, loss = 0.59035947
Iteration 15, loss = 0.58772058
Iteration 16, loss = 0.58151265
Iteration 17, loss = 0.59271009
Iteration 18, loss = 0.58689395
Iteration 19, loss = 0.57984969
Iteration 20, loss = 0.58531268
Iteration 21, loss = 0.58286370
Iteration 22, loss = 0.58269038
Iteration 23, loss = 0.57683059
Iteration 24, loss = 0.57704025
Iteration 25, loss = 0.57622863
Iteration 26, loss = 0.57861961
Iteration 27, loss = 0.57732753
Iteration 28, loss = 0.57530933
Iteration 29, loss = 0.57812743
Iteration 30, loss = 0.57262509
Iteration 31, loss = 0.57406335
Iteration 32, los

In [90]:
variables = ['summonerLevel', 'championLevel','championPoints', 
             'Assassin', 'Fighter', 'Mage', 'Marksman', 'Support', 'Tank',
            'info.attack', 'info.defense', 'info.magic', 'info.difficulty',
            'stats.hpregenperlevel',	'stats.mpregen', 'stats.mpregenperlevel',	'stats.crit',	'stats.critperlevel',
            'stats.attackdamage', 'stats.attackdamageperlevel', 'stats.attackspeedperlevel',	'stats.attackspeed',
            'final_gold', 'final_xp', 'final_abilityhaste', 'final_abilitypower', 'final_armor', 'final_armorpen',
            'final_armorpenpercent', 'final_atkdmg', 'final_bns_armorpenpercent', 'final_bns_magicpenpercent', 'final_ccreduction',
            'final_cdreduction', 'final_remaining_health', 'final_health', 'final_healthrgn', 'final_lifesteal', 'final_mppen',
            'final_mgpenpercent', 'final_mgres', 'final_ms', 'final_omnivamp', 'final_physicalvamp', 'final_power', 'final_powermax',
            'final_powerregen', 'final_spellvamp', 'final_currentgold', 'final_magicdmgdone', 'final_magicdmgdonetochamps', 'final_magicdmgtaken',
            'final_physdmgdone', 'final_physdmgdonetochamps', 'final_physdmgtaken', 'final_dmgdone', 'final_dmgdonetochamps', 'final_dmgtaken', 
            'final_truedmgdone', 'final_truedmgdonetochamps', 'final_truedmgtaken', 'final_goldpersec', 'final_jungleminionskilled', 'final_lvl',
            'final_minionskilled', 'final_jungleminionskilled', 'final_jungleminionskilled', 'final_jungleminionskilled', 'final_enemycontrolled'                  
             ]

X_test = pd.merge(X_test_original, team_positions, how='inner', on=['matchId', 'participantId'])
X_test = pd.merge(X_test, champion_data, how='inner', left_on='championId', right_on='key')
X_test = pd.merge(X_test, champion_types, how='inner', left_on='championId', right_on='key')
X_test = pd.merge(X_test, test_last_frame_values, how='inner', on=['matchId', 'participantId'])
X_test = pd.merge(X_test, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X_test = X_test.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)


convert_team_values(X_test, variables)

X_test_perlane = X_test.groupby(['matchId', 'teamPosition'])[['final_gold']].sum().pivot_table(values='final_gold', index='matchId', columns='teamPosition').reset_index().drop(columns=0)

for lane in ['BOTTOM', 'JUNGLE', 'MIDDLE', 'TOP', 'UTILITY']:
  X_test_perlane[f'{lane}'] = np.where(X_test_perlane[f'{lane}'] >= 0, 1, -1)

X_test = (
    X_test
    .groupby(['matchId'])[variables]
    .sum()
    .reset_index()
)

X_test = pd.merge(X_test, X_test_perlane, how='inner', on='matchId').reset_index(drop = True)
X_test = pd.merge(X_test, testing_events, how='inner', on='matchId').reset_index(drop = True)

In [91]:
y_pred = pipeline_neuralnetwork.predict(X_test)

In [92]:
submission['winner'] = y_pred
submission.head()

,matchId,winner
0,8000,100
1,8001,100
2,8002,100
3,8003,100
4,8004,200


In [93]:
submission.to_csv('../data/submission_neural_network_hidden_layers4x100_alpha005_activtahn_2023_03_30.csv', index=False)